In [33]:
import requests
import zipfile
import pandas as pd
from io import BytesIO

# Define the URL of the zip file
zip_file_url = 'https://github.com/skoltech-nlp/detox/releases/download/emnlp2021/filtered_paranmt.zip'

# Send an HTTP GET request to download the zip file
response = requests.get(zip_file_url)

# Check if the request was successful
if response.status_code == 200:
    # Read the zip file
    with zipfile.ZipFile(BytesIO(response.content)) as zip_file:
        # Assuming there's only one .tsv file in the zip, you can read it like this
        tsv_file = zip_file.namelist()[0]
        with zip_file.open(tsv_file) as tsv:
            # Read the .tsv file into a DataFrame
            train_text = pd.read_csv(tsv, sep='\t')

            # Display the head of the DataFrame
            head = train_text.head()
            print(head)
else:
    print("Failed to download the zip file.")

   Unnamed: 0                                          reference  \
0           0  If Alkar is flooding her with psychic waste, t...   
1           1                          Now you're getting nasty.   
2           2           Well, we could spare your life, for one.   
3           3          Ah! Monkey, you've got to snap out of it.   
4           4                   I've got orders to put her down.   

                                         translation  similarity  lenght_diff  \
0  if Alkar floods her with her mental waste, it ...    0.785171     0.010309   
1                        you're becoming disgusting.    0.749687     0.071429   
2                      well, we can spare your life.    0.919051     0.268293   
3                       monkey, you have to wake up.    0.664333     0.309524   
4                         I have orders to kill her.    0.726639     0.181818   

    ref_tox   trn_tox  
0  0.014195  0.981983  
1  0.065473  0.999039  
2  0.213313  0.985068  
3  0.053

In [34]:
head

,Unnamed: 0,reference,translation,similarity,lenght_diff,ref_tox,trn_tox
0,0,"If Alkar is flooding her with psychic waste, t...","if Alkar floods her with her mental waste, it ...",0.785171,0.010309,0.014195,0.981983
1,1,Now you're getting nasty.,you're becoming disgusting.,0.749687,0.071429,0.065473,0.999039
2,2,"Well, we could spare your life, for one.","well, we can spare your life.",0.919051,0.268293,0.213313,0.985068
3,3,"Ah! Monkey, you've got to snap out of it.","monkey, you have to wake up.",0.664333,0.309524,0.053362,0.994215
4,4,I've got orders to put her down.,I have orders to kill her.,0.726639,0.181818,0.009402,0.999348


In [35]:
from sklearn.model_selection import train_test_split

X = train_text.reference

y = train_text['ref_tox']

X_train, X_val, y_train, y_val = train_test_split(X, y, shuffle=True, random_state=123)

In [36]:
from sklearn.feature_extraction import _stop_words
import string

stop_words = _stop_words.ENGLISH_STOP_WORDS

def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    text_without_punct = text.translate(translator)
    return text_without_punct

def cleaning(doc):
    # Remove punctuation
    doc = remove_punctuation(doc)

    doc = ''.join([word for word in doc if not word.isdigit()])

    # Remove stop_words
    tokens = doc.split()
    cleaned_tokens = [token for token in tokens if token not in stop_words]
    cleaned_doc = ' '.join(cleaned_tokens)

    return cleaned_doc

In [37]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(max_features=10000, preprocessor=cleaning)
X_train_bow = vect.fit_transform(X_train)
X_val_bow = vect.fit_transform(X_val)

In [38]:
pd.DataFrame(X_train_bow.A[:5], columns=vect.get_feature_names_out())

,AIDS,ALL,AM,AND,ARCHER,Aah,Aaron,Abby,About,Abraham,...,youve,zap,zealot,zero,zip,zombie,zombies,zone,zoo,Ηe
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
import numpy as np
# Set a threshold for binary classification
threshold = 0.5

# Convert to binary labels
y_train_binary = np.array([1 if val >= threshold else 0 for val in y_train])

# Alternatively, using NumPy
y_train_binary = (np.array(y_train) >= threshold).astype(int)

print(y_train_binary)

[1 1 0 ... 1 1 0]


In [45]:
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(max_iter = 30000)
classifier.fit(X_train_bow, y_train_binary)

LogisticRegression(max_iter=30000)

In [46]:
X_val_bow = vect.fit_transform(X_val)

In [47]:
# Alternatively, using NumPy
y_val_binary = (np.array(y_val) >= threshold).astype(int)

y_pred = classifier.predict(X_val_bow)
accuracy = accuracy_score(y_val_binary, y_pred)
mse = mean_squared_error(y_val_binary, y_pred)
accuracy

0.5295718093391949

In [70]:
feat_impts = [vect.get_feature_names_out()]

lr_feature_importances = classifier.coef_.flatten()

# Saving the results in a DataFrame
df_feats_impts = pd.DataFrame({"word": feat_impts[0], "feature_importance": lr_feature_importances})

# Converting the feature importance column from string to float
df_feats_impts["feature_importance"] = df_feats_impts["feature_importance"].astype("float32")

df_feats_impts.head()

,word,feature_importance
0,AIDS,0.303347
1,ALL,-0.389954
2,AM,0.973200
3,AND,-0.619881
4,ARCHER,0.791435


In [74]:
sorted_df = df_feats_impts.sort_values(by="feature_importance", ascending=False)
sorted_df.head(20)

,word,feature_importance
1779,Shit,7.267882
477,Dana,7.178648
5135,glued,7.144665
737,Friday,7.037260
8277,shithead,6.958261
2550,assassins,6.718610
7050,pattern,6.579329
3095,bum,6.534000
2835,bitching,6.522133
4983,frozen,6.514779
